This notebook is aimed at training a pre-trained GPT-2 model in order to learn how to generate lyrics. The training corpus was gathered scraping lyrics of artists known for being rather pessimistic, melancholic or even hateful. The goal is to train GPT-2 in learning this style of lyrics and then use PPLM to conditional genering lyrics with opposite style.

In [ ]:
%%capture
!pip install transformers==4.11.3
!pip install datasets

In [3]:
import json
import os
import pandas as pd
import torch
import re
import pickle
from sklearn.model_selection import train_test_split
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
lyrics = []
segnibrutti = ["URLCopyEmbedCopy", '\d+EmbedShare', 'EmbedShare', ";"] #get rid of strange symbols in the dataset
lista_canz = [] 
for filename in os.listdir("/content/drive/MyDrive/Colab_Notebooks/lyrics_generator/lyrics/General"): 
  if ".json" in filename:
    with open("/content/drive/MyDrive/Colab_Notebooks/lyrics_generator/lyrics/General/"+filename, encoding='utf-8') as json_file:
      data = json.load(json_file)
      for track in data["songs"]:
        canz = track["title"]
        lyric = track["lyrics"]
        #lyric = lyric.replace("\n", " ")
        for segno in segnibrutti:
          lyric = re.sub(segno.lower(), "", lyric.lower())
        lyric =  re.sub('\n{2,}',  '\n', lyric) #clear more than 1 linebreak
        counter = 0
        for car in lyric:
          if car == "\n":
            counter += 1
            if counter > len(lyric) / 15:
              lista_canz.append(canz)  
        #sent_score = sentiment_analyzer_scores(lyric)
        if canz not in lista_canz and lyric != "" and not ("demo" in canz.lower() or "live" in canz.lower() or "version" in canz.lower() or 
                                           "session" in canz.lower() or "edit" in canz.lower() or "mix" in canz.lower()):# and float(sent_score["compound"]) < 0.97: #filter for outliers 
          lista_canz.append(canz)
          lyrics.append(lyric)

train, test = train_test_split(lyrics,test_size=0.20, shuffle=True)
df_train = pd.DataFrame(train, columns=["text"])
df_test= pd.DataFrame(test, columns=["text"])
df_train.to_csv("train.csv", index=False, sep=';')
df_test.to_csv("test.csv", index=False, sep=';' )
print(len(train), len(test))
print(lista_canz)

4640 1160
['Acid Bubble', 'Again', 'A Little Bitter', 'All I Am', 'All Secrets Known', 'A Looking in View', 'Am I Inside', 'Angry Chair', 'Angry Chair (Unplugged)', 'Black Gives Way to Blue', 'Bleed the Freak', 'Breath on a Window', 'Brother', 'Brother (Unplugged)', 'Brush Away', 'Check My Brain', 'Chemical Addiction', 'Choke', 'Confusion', 'Dam That River', 'Deaf Ears Blind Eyes', 'Died', 'Dirt', 'Don’t Be Satisfied', 'Don’t Follow', 'Down in a Hole', 'Down in a Hole (Unplugged)', 'Drone', 'Fairy Tale Love Story', 'Fat Girls', 'Fear the Voices', 'Fly', 'Football', 'Frogs', 'Frogs (Unplugged)', 'Get Born Again', 'Glamorous Girls', 'God Am', 'God Smack', 'Got Me Wrong', 'Got Me Wrong (Unplugged)', 'Grind', 'Hate to Feel', 'Head Creeps', 'Heaven Beside You', 'Heaven Beside You (Unplugged)', 'Hollow', 'Hung on a Hook', 'Hush Hush', 'I Can’t Have You Blues', 'I Can’t Remember', 'I Know Somethin’ (’Bout You)', 'Iron Gland', 'I Stay Away', 'It Ain’t Like That', 'Junkhead', 'Killer Is Me', 'K

In [ ]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files={'train': 'train.csv','test': 'test.csv'}, sep = ";")
def tokenize_function(example):
    return tokenizer([a + "\n" for a in example["text"]], truncation=True, padding=True) #new lines

Using custom data configuration default-96da0c96d3b399b2


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-96da0c96d3b399b2/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
print(dataset["train"]["text"])
print(dataset["train"]["text"][43])
print(dataset)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification,DataCollatorForLanguageModeling

tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [ ]:
tokenized_datasets = dataset.map(tokenize_function, batched=True,  remove_columns=["text"])
#columns_to_return = ['input_ids', 'attention_mask']
#tokenized_datasets.set_format(type='torch', columns=columns_to_return)

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
print(tokenizer.convert_ids_to_tokens(tokenized_datasets["train"]["input_ids"][40]))
print(tokenizer.convert_ids_to_tokens(tokenized_datasets["train"]["input_ids"][50]))
print(tokenized_datasets)

['[', 'verse', ']', 'Ċ', 'mid', 'night', 'Ġin', 'Ġthe', 'Ġsubway', ',', 'Ġshe', "'s", 'Ġon', 'Ġher', 'Ġway', 'Ġhome', 'Ċ', 'she', 'Ġtries', 'Ġhard', 'Ġnot', 'Ġto', 'Ġrun', 'Ġbut', 'Ġshe', 'Ġfeels', 'Ġshe', "'s", 'Ġnot', 'Ġalone', 'Ċ', 'ech', 'oes', 'Ġof', 'Ġfootsteps', 'Ġfollow', 'Ġclose', 'Ġbehind', 'Ċ', 'but', 'Ġshe', 'Ġdare', 'Ġnot', 'Ġturn', 'Ġaround', 'Ċ', 'turn', 'Ġaround', 'Ċ', 'turn', 'Ġaround', 'Ġ', 'Ċ', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|endoftext|>', '<|en

In [ ]:
block_size = 128
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

Also note that by default, the map method will send a batch of 1,000 examples to be treated by the preprocessing function. So here, we will drop the remainder to make the concatenated tokenized texts a multiple of block_size every 1,000 examples. You can adjust this behavior by passing a higher batch size (which will also be processed slower). You can also speed-up the preprocessing by using multiprocessing:

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)
lm_datasets.set_format(type='torch')

In [ ]:
print(lm_datasets)

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'labels'],
        num_rows: 37120
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'labels'],
        num_rows: 9280
    })
})


In [ ]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead
model = AutoModelWithLMHead.from_pretrained("gpt2", resid_pdrop=0.7) #dropout to avoid overfitting
embedding_layer = model.resize_token_embeddings(len(tokenizer))


/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:664: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

In [ ]:
print(len(tokenizer))

50257


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("/content/drive/MyDrive/Colab_Notebooks/lyrics_generator/lyrics/General/training/final",
  overwrite_output_dir = True,
  logging_first_step = True,
  num_train_epochs = 10,
  warmup_steps = 200,
  per_device_train_batch_size=24, 
  per_device_eval_batch_size=24,
  evaluation_strategy = "steps",
  logging_steps = 250,
  save_steps = 500,
  save_total_limit = 5,
  gradient_accumulation_steps = 4,
  learning_rate = 5e-4,
  weight_decay = 0.4
  
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer
)

In [ ]:
trainer.train() #resume_from_checkpoint="/content/drive/MyDrive/Colab_Notebooks/lyrics_generator/lyrics/General/training/checkpoint-1500" #restart from checkpoint

***** Running training *****
  Num examples = 37120
  Num Epochs = 10
  Instantaneous batch size per device = 24
  Total train batch size (w. parallel, distributed & accumulation) = 96
  Gradient Accumulation steps = 4
  Total optimization steps = 3860


Step,Training Loss,Validation Loss
250,5.216800,3.990979
500,3.960700,3.632290
750,3.660300,3.520120
1000,3.449300,3.489848
1250,3.392400,3.456037
1500,3.282100,3.423117
1750,3.183100,3.417938
2000,3.157100,3.401701


***** Running Evaluation *****
  Num examples = 9280
  Batch size = 24
***** Running Evaluation *****
  Num examples = 9280
  Batch size = 24
Saving model checkpoint to /content/drive/MyDrive/Colab_Notebooks/lyrics_generator/lyrics/General/training/final/checkpoint-500
Configuration saved in /content/drive/MyDrive/Colab_Notebooks/lyrics_generator/lyrics/General/training/final/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/Colab_Notebooks/lyrics_generator/lyrics/General/training/final/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Colab_Notebooks/lyrics_generator/lyrics/General/training/final/checkpoint-500/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Colab_Notebooks/lyrics_generator/lyrics/General/training/final/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 9280
  Batch size = 24
***** Running Evaluation *****
  Num examples = 9280
  Batch size = 24
Sa

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 6080
  Batch size = 32


Perplexity: 23.91


In [ ]:
print(eval_results)

NameError: ignored

In [ ]:
history = trainer.state.log_history
print(history)

steps = []
train_loss = []
eval_loss = []
i = 0
while i <= len(history) - 5:
  if i % 2 != 0:
    eval_loss.append(history[i]["eval_loss"])
  else:
    train_loss.append(history[i]["loss"])
    steps.append(history[i]["step"])
  i = i +1

import matplotlib.pyplot as plt

#history for accuracy
plt.plot(train_loss)
plt.plot(eval_loss)
plt.title('Perplexity: '+ str(math.exp(eval_results['eval_loss'])))
plt.ylabel('accuracy')
plt.xlabel('steps')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig("General_model_accuracy_gpt2.png")
plt.show()

AttributeError: ignored

In [ ]:
trainer.save_model("/content/drive/MyDrive/Colab_Notebooks/lyrics_generator/lyrics/General/model")
tokenizer.save_pretrained('/content/drive/MyDrive/Colab_Notebooks/lyrics_generator/lyrics/General/model')


Saving model checkpoint to /content/drive/MyDrive/Colab_Notebooks/lyrics_generator/lyrics/General/model
Configuration saved in /content/drive/MyDrive/Colab_Notebooks/lyrics_generator/lyrics/General/model/config.json
Model weights saved in /content/drive/MyDrive/Colab_Notebooks/lyrics_generator/lyrics/General/model/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Colab_Notebooks/lyrics_generator/lyrics/General/model/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Colab_Notebooks/lyrics_generator/lyrics/General/model/special_tokens_map.json
tokenizer config file saved in /content/drive/MyDrive/Colab_Notebooks/lyrics_generator/lyrics/General/model/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Colab_Notebooks/lyrics_generator/lyrics/General/model/special_tokens_map.json


('/content/drive/MyDrive/Colab_Notebooks/lyrics_generator/lyrics/General/model/tokenizer_config.json',
 '/content/drive/MyDrive/Colab_Notebooks/lyrics_generator/lyrics/General/model/special_tokens_map.json',
 '/content/drive/MyDrive/Colab_Notebooks/lyrics_generator/lyrics/General/model/vocab.json',
 '/content/drive/MyDrive/Colab_Notebooks/lyrics_generator/lyrics/General/model/merges.txt',
 '/content/drive/MyDrive/Colab_Notebooks/lyrics_generator/lyrics/General/model/added_tokens.json',
 '/content/drive/MyDrive/Colab_Notebooks/lyrics_generator/lyrics/General/model/tokenizer.json')

In [4]:
from transformers import pipeline
lyrics_generation = pipeline('text-generation',model='/content/drive/MyDrive/Colab_Notebooks/lyrics_generator/lyrics/General/training/ultimaprova', max_length=200, num_return_sequences=2)

In [5]:
lyrics = lyrics_generation("[verse 1]", max_length=150, do_sample=True, temperature = 0.8, num_return_sequences=2)
text1 = lyrics[0]["generated_text"]
text2= lyrics[1]["generated_text"]

print(text1, "\n\n---------------\n\n", text2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[verse 1]
i remember someone who spoke in tongues
i remember when, dear mary said
"i never loved you."
and the day i met her
said "i never loved you
and the day i met her
"i never loved you."
and the day i met her
said "i never loved you"
and i never loved you
and the day i met her
said "i never loved you" 
[chorus]
and you said "i never loved you
and the day i met her
and you said "i never loved you"
and you said "i never loved you
and the day i met her
and you said "i never loved you" 

---------------

 [verse 1]
the next artery came too soon
i saw you standing still
you were standing still
with the same old guitar that you had
oh, i thought i'd love you
but you were standing still
with the same old guitar that you had
oh, i thought i'd love you
but you were standing still, with the same old guitar that you had
oh, i thought i'd love you
[chorus]
you were standing still, with the same old guitar that you had
oh, i thought i'd love you
but you were standing still
with the same old gu

In [6]:
lyrics = lyrics_generation("[verse 1]", max_length=150, do_sample=True, temperature = 1.0, num_return_sequences=2)
text1 = lyrics[0]["generated_text"]
text2= lyrics[1]["generated_text"]

print(text1, "\n\n---------------\n\n", text2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[verse 1]
i'm going to heaven to be alright
i'm going to heaven to be alright
my kingdom was always devout
all i ever wanted was god to be alright
but that's not heaven to be alright
[chorus]
and the devil came to heaven
when he walked away from the dark
[verse 2]
i got out of love with you
if there's a devil, i can't stop
the devil came to heaven
when he walked away from the dark
god came to heaven
god came to heaven
when he walked away from the dark
[chorus]
and the devil came to heaven
when he walked away from the dark
and the dark
when he 

---------------

 [verse 1]
someone said i will never do anything
you know you don't know what you are
you know what you're
you know i will never do anything
you know i will never do anything
you know what you are
[verse 2]
someone said i will never do anything
you know i will never do anything
you know i will never do anything
you know i will never do anything
you know i will never do anything
you know i will never do anything
[chorus]
you alwa

In [7]:
lyrics = lyrics_generation("[verse 1]", max_length=150, do_sample=True, temperature = 1.2, num_return_sequences=2)
text1 = lyrics[0]["generated_text"]
text2= lyrics[1]["generated_text"]

print(text1, "\n\n---------------\n\n", text2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[verse 1]
they were talking about a new wave
well when i first spoke of you
i thought that it would be me
well when it had no right to call you to me
because i was in the back age of summer
[chorus 1]
my parents were praying, 'we can call our love again.'
but i wanted to hold
[verse 2]
the kids had consoling me
the children did not mind the boys
and even though she knew how
and when they cried about a new wave
[chorus 2]
my parents were praying, 'we can call our love again.'
because i didn't know how
and now they sang of a new wave
[ 

---------------

 [verse 1]
i am the one who can have been
and there's one who can've been
[verse 2]
there is one who sits out and laughs
there's one who will hear her voice
there must be someone crying
there must be someone dancing on the ground
[chorus]
yeah, all this yelling is silence
because she's not even going crazy
and she's not really going crazy
[verse 3]
she's a crying
about walking on the ground
yeah, not really going crazy
but she don't wann

In [8]:
lyrics = lyrics_generation("[verse 1]", max_length=150, do_sample=True, temperature = 1.4, num_return_sequences=2)

text1 = lyrics[0]["generated_text"]
text2= lyrics[1]["generated_text"]

print(text1, "\n\n---------------\n\n", text2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[verse 1]
sometimes time there comes sunlight
and the stars will glow
i'll remember the things that you remember
and that make you sleep
i'll remember things that i saw
i remember the nights you made you cry
you kept playing out loud
on the walls you've gone
[pre-chorus 1]
the dream you dreamed
you had run away from home
and said "hey, but i wouldn't say no"
because you made all up your words
because you put things back up
with all these people you had gone
did they all know for you? did them all know for you? did they all know for you? did they do? 
did i remember the things 

---------------

 [verse 1]
is filled with blood for a woman with lungs
then she'd rather be more than the sun
though all he wanted her to sing
he's too beautiful when you see, he loves anything you do
than she can get you, so she wants to play as you love
[chorus 1]
o, o me and now i have to say (ooh)
o, o me and now i have to say, oh
o, o me and now i have to say (ooh)
o, o me and now i have to say (ooh) 
o, o

In [9]:
lyrics = lyrics_generation("[verse 1]", max_length=150, do_sample=True, temperature = 1.6, num_return_sequences=2)
text1 = lyrics[0]["generated_text"]
text2= lyrics[1]["generated_text"]

print(text1, "\n\n---------------\n\n", text2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[verse 1]
it feels alright like a drunken storm, then is me
just so good about remembering that bad weather didn't come and didn't come around
to make the sad wind come so good 
every single word i say and make no sound
[chorus]
in love i tell you what you found is fine as me
i tell you that i won't miss her
sometimes when my hands hurt them now, i miss them now, well i miss them again. never really liked everything you gave in a dream
there are nothing to change with me and you better have it here too late
there are nothing to change with me but i love the ones you've sold but i did not think i love her 

---------------

 [verse 1]
the world has gone, oh yes
oh oh yeah yeahoh yeah that the world has gone, oh well
i'm falling, down with blood pouring down my throat
i look through his eyes, open his mouth
and it's like a vampire in the eye
when you put him in his bag, let him be
[chorus 1]
it's hard, hard not to explain
i don't believe with you
you're losing me
for crying, crying now, 

In [10]:
lyrics = lyrics_generation("[verse 1]", max_length=150, do_sample=True, temperature = 1.8, num_return_sequences=2)

text1 = lyrics[0]["generated_text"]
text2= lyrics[1]["generated_text"]

print(text1, "\n\n---------------\n\n", text2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[verse 1]
now this is not our destiny
all of this is wrong
my face still burns black, in the light of hell
on how many streets have closed
from eachwhere nobody goes
to take their toll or lose control is
a place where nobody understands
and even after their voices speak too harshly before us fall apart, it always will fade away
'cause this is not our lifetime, the end is all our life's ours
my whole world's nothing ever wrong at all.'
[chorus]
in time i walk all night
through the dark and in the dark when i get a little closer too deep in
i am leaving alone 
a place where they do, i'll start walking 

---------------

 [verse 1]
hey little boys can say maybe not much about your mother even
but i feel ashamed about the girl so much and lovely and so handsome, how you feel i would
when i was sixty years old with one, all together again i want to forget when i was young
and i would ask her about "go home tonight"? and let you leave it
oh dear, why can i still feel this at the same
"can i 